# Logging results and uploading models to Weights & Biases
In this example, we train a simple XGBoost model and log the training
results to Weights & Biases. We also save the resulting model checkpoints
as artifacts.

Let's start with installing our dependencies:

In [1]:
!pip install -qU "ray[tune]" sklearn xgboost_ray wandb

Then we need some imports:

In [2]:
import ray

from ray.ml import RunConfig
from ray.ml.result import Result
from ray.ml.train.integrations.xgboost import XGBoostTrainer
from ray.tune.integration.wandb import WandbLoggerCallback
from sklearn.datasets import load_breast_cancer

We define a simple function that returns our training dataset as a Ray Dataset:


In [3]:
def get_train_dataset() -> ray.data.Dataset:
    """Return the "Breast cancer" dataset as a Ray dataset."""
    data_raw = load_breast_cancer(as_frame=True)
    df = data_raw["data"]
    df["target"] = data_raw["target"]
    return ray.data.from_pandas(df)

Now we define a simple training function. All the magic happens within the `WandbLoggerCallback`:

```python
WandbLoggerCallback(
    project=wandb_project,
    save_checkpoints=True,
)
```

It will automatically log all results to Weights & Biases and upload the checkpoints as artifacts. It assumes you're logged in into Wandb via an API key or `wandb login`.

In [4]:
def train_model(train_dataset: ray.data.Dataset, wandb_project: str) -> Result:
    """Train a simple XGBoost model and return the result."""
    trainer = XGBoostTrainer(
        scaling_config={"num_workers": 2},
        params={"tree_method": "auto"},
        label_column="target",
        datasets={"train": train_dataset},
        num_boost_round=10,
        run_config=RunConfig(
            callbacks=[
                # This is the part needed to enable logging to Weights & Biases.
                # It assumes you've logged in before, e.g. with `wandb login`.
                WandbLoggerCallback(
                    project=wandb_project,
                    save_checkpoints=True,
                )
            ]
        ),
    )
    result = trainer.fit()
    return result

Let's kick off a run:

In [5]:
wandb_project = "ray_air_example"

train_dataset = get_train_dataset()
result = train_model(train_dataset=train_dataset, wandb_project=wandb_project)

2022-05-19 15:22:11,956	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8266
2022-05-19 15:22:15,995	INFO wandb.py:172 -- Already logged into W&B.


Trial name,status,loc,iter,total time (s),train-rmse
XGBoostTrainer_14a73_00000,TERMINATED,127.0.0.1:20065,10,10.2724,0.030717


(raylet) 2022-05-19 15:22:17,422	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/coding/ray/python/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61838 --object-store-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=63609 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:62933 --redis-password=5241590000000000 --startup-token=16 --runtime-env-hash=-2010331134
wandb: Currently logged in as: kaifricke. Use `wandb login --relogin` to force relogin
(GBDTTrainable pid=20065) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(raylet) 2022-05-19 15:22:23,215	INFO context.py:70 -- Exec'

(GBDTTrainable pid=20065) 2022-05-19 15:22:24,711	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(raylet) 2022-05-19 15:22:26,090	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/coding/ray/python/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61838 --object-store-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=63609 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:62933 --redis-password=5241590000000000 --startup-token=18 --runtime-env-hash=-2010331069
(raylet) 2022-05-19 15:22:26,234	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/cod

Result for XGBoostTrainer_14a73_00000:
  date: 2022-05-19_15-22-31
  done: false
  experiment_id: 2d50bfe80d2a441e80f4ca05f7c3b607
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 20065
  should_checkpoint: true
  time_since_restore: 10.080440044403076
  time_this_iter_s: 10.080440044403076
  time_total_s: 10.080440044403076
  timestamp: 1652970151
  timesteps_since_restore: 0
  train-rmse: 0.357284
  training_iteration: 1
  trial_id: 14a73_00000
  warmup_time: 0.006903171539306641
  


wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000001)... Done. 0.1s
(GBDTTrainable pid=20065) 2022-05-19 15:22:32,051	INFO main.py:1519 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.37 seconds (2.79 pure XGBoost training time).
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000002)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000003)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000004)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XG

Result for XGBoostTrainer_14a73_00000:
  date: 2022-05-19_15-22-32
  done: true
  experiment_id: 2d50bfe80d2a441e80f4ca05f7c3b607
  experiment_tag: '0'
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 20065
  should_checkpoint: true
  time_since_restore: 10.272444248199463
  time_this_iter_s: 0.023891210556030273
  time_total_s: 10.272444248199463
  timestamp: 1652970152
  timesteps_since_restore: 0
  train-rmse: 0.030717
  training_iteration: 10
  trial_id: 14a73_00000
  warmup_time: 0.006903171539306641
  


2022-05-19 15:22:42,727	INFO tune.py:753 -- Total run time: 27.83 seconds (26.61 seconds for the tuning loop).


iterations_since_restore,▁▂▃▃▄▅▆▆▇█
time_since_restore,▁▂▃▃▄▅▅▆▇█
time_this_iter_s,█▁▁▁▁▁▁▁▁▁
time_total_s,▁▂▃▃▄▅▅▆▇█
timestamp,▁▁▁▁▁▁▁▁██
timesteps_since_restore,▁▁▁▁▁▁▁▁▁▁
train-rmse,█▆▄▃▂▂▂▁▁▁
training_iteration,▁▂▃▃▄▅▆▆▇█
warmup_time,▁▁▁▁▁▁▁▁▁▁
iterations_since_restore,10
time_since_restore,10.27244


Check out your [WandB](https://wandb.ai/) project to see the results!